In [1]:
import numpy as np
from collections import defaultdict
import random

<h3>1. Build classes to store data:</h3>

In [2]:
#store the data of each member 
class Member:
    def __init__(self, r_d, label = None, doc_id = None):
        self._r_d = r_d
        self._label = label
        self._doc_id = doc_id
#class Cluster
class Cluster:
    def __init__(self):
        self._centroid = None
        self._members=[]
    #set centroid
    def set_centroid(self,new_centroid):
        self._centroid = new_centroid
    #reset list of members
    def reset_members(self):
        self._members = []
    #add new member
    def add_member(self,member):
        self._members.append(member)

<h3>2. Build class Kmeans:</h3>

In [3]:
class Kmeans:
    
    def __init__(self,num_clusters):
        self._num_clusters = num_clusters
        self._clusters = [Cluster() for _ in range(self._num_clusters)] #list of clusters
        self._E = [] #list of centroids
        self._S = 0 #overall similarity
    
    #load data
    def load_data(self, data_path="/Users/Admin/DSLab Training sessions/Session_2/data/"):
        #get the list of tfidf values of a sparse_r_d
        def sparse_to_dense(sparse_r_d,vocab_size):
            r_d = [0.0 for _ in range(vocab_size)]
            indices_tfidfs = sparse_r_d.split()
            #store tfidf values in initialized r_d
            for index_tfidf in indices_tfidfs:
                index = int(index_tfidf.split(':')[0])
                tfidf = float(index_tfidf.split(':')[1])
                r_d[index] = tfidf
            return np.array(r_d)
        
        #read data and vocab_size
        with open(data_path + "data_tf_idf.txt") as f:
            d_lines = f.read().splitlines()
        with open(data_path + "words_idfs.txt") as f:
            vocab_size = len(f.read().splitlines())
        #store tf-idf, label of newsgroup and file names
        self._data = []
        #store number of files in newsgroup
        self._label_count = defaultdict(int)
        #append data point to self._data
        for data_id, d in enumerate(d_lines):
            features = d.split('<fff>')
            label, doc_id = int(features[0]), int(features[1])
            self._label_count[label]+=1
            r_d = sparse_to_dense(sparse_r_d = features[2], vocab_size=vocab_size)
            #append member with class Member
            self._data.append(Member(r_d=r_d,label=label,doc_id=doc_id))
    #initialize centroid in a random way
    def random_init(self,seed_value):
        random.seed(seed_value)
        #get the list of data points
        members = [member._r_d for member in self._data]
        #return a random array having length num_clusters, elements are less than len(self._data)
        pos = np.random.choice(len(self._data), self._num_clusters, replace=False)
        centroids=[]
        for i in pos:
            centroids.append(members[i])
        self._E = centroids
        # update centroid
        for i in range(self._num_clusters):
            self._clusters[i].set_centroid(centroids[i])
    # using cosine similarity: S(x,y) = x.y / |x|*|y|       
    def compute_similarity(self, member, centroid):
        return centroid.dot(member._r_d) * 1. / np.linalg.norm(centroid)*np.linalg.norm(member._r_d)
    #select cluster for a data point
    def select_cluster_for(self,member):
        best_fit_cluster = None
        #initialize max_similarity: cos(x)=-1 => x=180
        max_similarity = -1
        #search in set of clusters
        for cluster in self._clusters:
            #calculate the similarity between data point and centroid
            similarity = self.compute_similarity(member,cluster._centroid)
            if similarity > max_similarity:
                best_fit_cluster = cluster
                max_similarity = similarity
        #add data point to cluster
        best_fit_cluster.add_member(member)
        return max_similarity
    #update centroid of a cluster
    def update_centroid_of(self,cluster):
        #crawl list of tfidf vector
        member_r_ds = [member._r_d for member in cluster._members]
        #calculate the new centroid
        aver_r_d = np.mean(member_r_ds, axis=0)
        sqrt_sum_sqr = np.sqrt(np.sum(aver_r_d ** 2))
        new_centroid = np.array([value/sqrt_sum_sqr for value in aver_r_d])
        #update new centroid
        cluster._centroid = new_centroid
    #check for stopping condition
    def stopping_condition(self, criterion, threshold):
        #list of criteria
        criteria = ['max_iters','similarity','centroid']
        assert criterion in criteria
        #check conditions of each criterion
        if criterion == 'max_iters':
            if self._iterations >= threshold:
                return True
            else:
                return False
        elif criterion == 'centroid':
            E_new = [list(cluster._centroid) for cluster in self._clusters]
            E_new_minus_E = [centroid for centroid in E_new if centroid not in self._E]
            self._E = E_new
            if len(E_new_minus_E) < threshold:
                return True
            else:
                return False
        else:
            new_S_minus_S = self._new_S - self._S
            self._S = self._new_S
            if new_S_minus_S < threshold:
                return True
            else:
                return False
    def run(self, seed_value, criterion, threshold):
        #randomly initialize set of centroids and number of iteration
        self.random_init(seed_value)
        self._iteration = 0
        #run process
        while True:
            #reset members of each cluster
            for cluster in self._clusters:
                cluster.reset_members()
            self._new_S = 0
            #choose cluster for each member
            for member in self._data:
                max_s = self.select_cluster_for(member)
                self._new_S += max_s
            #determine new centroid of each cluster
            for cluster in self._clusters:
                self.update_centroid_of(cluster)
            
            self._iteration += 1
            #check for stopping condition
            if self.stopping_condition(criterion, threshold):
                break
    #compute purity value
    def compute_purity(self):
        majority_sum = 0
        for cluster in self._clusters:
            #set of label of members of cluster
            member_labels = [member._label for member in cluster._members]
            #maximum of numbers of data points of label, note: we have 20 labels
            max_count = max([member_labels.count(label) for label in range(20)])
            majority_sum+=max_count
        return majority_sum * 1. / len(self._data)
    #compute NMI value
    def compute_NMI(self):
        I_value, H_omega, H_C, N = 0. ,0. , 0. ,len(self._data)
        for cluster in self._clusters:
            wk = len(cluster._members) * 1.
            H_omega += -wk/N * np.log10(wk/N)
            member_labels = [member._label for member in cluster._members]
            for label in range(20):
                wk_cj = member_labels.count(label) * 1.
                cj = self._label_count[label]
                I_value += wk_cj / N * np.log10(N * wk_cj/(wk*cj) + 1e-12)
        for label in range(20):
            cj = self._label_count[label]
            H_C += -cj/N * np.log10(cj/N)
        return I_value * 2. / (H_omega + H_C)
        

<h3>3. Evaluation:</h3>

In [4]:
kmeans = Kmeans(num_clusters=8)
kmeans.load_data()
kmeans.run(seed_value=42, criterion='similarity', threshold=1e-3)
print("Purity:{} NMI:{}".format(kmeans.compute_purity(),kmeans.compute_NMI()))

Purity:0.3720152817574021 NMI:0.5303526484639328
